In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import pickle
import pprint
from io import StringIO
from flask import jsonify

%matplotlib inline

In [2]:
root_path = ''
df_title = pd.read_csv(root_path + 'data/soc-redditHyperlinks-title.tsv', sep='\t')
df_body = pd.read_csv(root_path + 'data/soc-redditHyperlinks-body.tsv', sep='\t')
df_points = pd.read_csv("data/projections/perp_10_2500subs.csv")
df_all = pd.concat([df_title, df_body])

all_subs = set()
all_subs.update(df_points['sub'])

df_all = df_all[df_all.SOURCE_SUBREDDIT.map(lambda x: x in all_subs) | df_all.TARGET_SUBREDDIT.map(lambda x: x in all_subs)]
df_all.TIMESTAMP = pd.to_datetime(df_all.TIMESTAMP)
df_all.sort_values(by='TIMESTAMP', inplace=True)
df_all.reset_index(inplace=True)

df_source = df_all.set_index('SOURCE_SUBREDDIT')
df_target = df_all.set_index('TARGET_SUBREDDIT')

In [3]:
def filter_dates(df, start, end):
    return 

def filter_subs():
    return

Linked To:
Linked From:

Net Received Sentiment
Net Sent Sentiment

Daily To Net Sentiment:
Daily Out Net Sentiment:

In [ ]:
links = df_all.groupby(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT', 'LINK_SENTIMENT'])['POST_ID'].count().reset_index()
links.columns = 'source target sentiment n'.split()
links = links.sort_values(by='n', ascending=False)

In [ ]:
link_limit = 2000


mask = links.sentiment == 1
pos_links = links[mask].iloc[:link_limit]
neg_links = links[~mask].iloc[:link_limit]

In [ ]:
def link_counts(df):
    mask = df.LINK_SENTIMENT == 1
    positive = df[mask]
    negative = df[~mask]
    
    return [positive.index.value_counts(), negative.index.value_counts()]

In [ ]:
# # def get_link_counts()
# def process_frame(df_points):
#     pcounts, ncounts = link_counts(df_target)
#     df_points['in_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
#     df_points['in_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))

#     pcounts, ncounts = link_counts(df_source)
#     df_points['out_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
#     df_points['out_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))
#     return df_points

In [ ]:
# def get_link_counts()
def process_frame(df_points):
    pcounts, ncounts = link_counts(df_target)
    df_points['in_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
    df_points['in_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))
    df_points['in_links'] = df_points['in_neg'] + df_points['in_pos']

    pcounts, ncounts = link_counts(df_source)
    df_points['out_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
    df_points['out_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))
    df_points['out_links'] = df_points['out_pos'] + df_points['out_neg']
    return df_points

In [4]:
bins = pd.cut(df_all.TIMESTAMP, 50)
df_all['bin'] = bins
pos_rows = df_all[df_all.LINK_SENTIMENT == 1]
neg_rows = df_all[df_all.LINK_SENTIMENT == -1]
df_vol = pd.DataFrame(pos_rows.groupby('bin')['POST_ID'].count().values, columns=['positive'])
df_vol['negative'] = neg_rows.groupby('bin')['POST_ID'].count().values

In [99]:
filtered_pos = bins.loc[df_all[df_all.LINK_SENTIMENT == 1].index]

In [103]:
%timeit df_all[df_all.LINK_SENTIMENT == 1]

121 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
%timeit df_all.loc[pos_rows]

170 ms ± 6.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [83]:
df_test = df_all.copy()

In [94]:
df_all.loc[23]

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,..."
23,corejerk,bestof2013,1u5w42s,2014-01-01 04:58:04,1,"2735.0,2451.0,0.722120658135,0.0197440585009,0..."


In [110]:
popos_rows.groupby('bin').count()['index'].values

(50,)

In [6]:
test = df_source.groupby(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT']).count()

In [7]:
test.shape

(302692, 5)

In [10]:
df_all.shape

(810702, 8)

In [11]:
df_body.shape[0] + df_title.shape[0]

858488

In [12]:
df_all.shape[0]

810702

In [49]:
link_limit = 2000
links = df_all.groupby(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT', 'LINK_SENTIMENT'])['POST_ID'].count().reset_index()
links.columns = 'source target sentiment n'.split()
links = links.sort_values(by='n', ascending=False)

mask = links.sentiment == 1
pos_links = links[mask].iloc[:link_limit]
neg_links = links[~mask].iloc[:link_limit]

In [63]:
def prepare_json(df):
    json_obj = StringIO()
    df.to_json(json_obj)
    json_ready = json_obj.getvalue()
    json_obj.close()
    return json_ready

In [66]:
test = prepare_json(df_points)

In [67]:
pprint.pprint(test)

'{"x":{"0":-58.47407,"1":102.64898,"2":108.12677,"3":-109.04855,"4":-48.705154,"5":-103.34196,"6":-102.798546,"7":124.939255,"8":91.24281,"9":-113.88309,"10":-58.14214,"11":86.33822,"12":-100.40617,"13":-100.68958,"14":106.97662,"15":101.80424,"16":-78.32568,"17":104.07287,"18":56.307114,"19":-2.4507651,"20":-91.13355,"21":108.13052,"22":-25.18722,"23":-36.99391,"24":-60.670033,"25":7.429544,"26":107.24084,"27":-25.87791,"28":103.82976,"29":-105.73568,"30":107.21719,"31":-92.98625,"32":-119.47189,"33":112.54999,"34":-122.469406,"35":-61.41778,"36":100.42506,"37":-107.391045,"38":81.23606,"39":-2.7350123,"40":-81.54253,"41":-111.291595,"42":142.5158,"43":-118.17944,"44":144.71027,"45":-12.410384,"46":6.033732,"47":-57.885788,"48":102.04873,"49":-44.989864,"50":116.26937,"51":-48.845726,"52":-52.754646,"53":101.5411,"54":-71.91384,"55":-67.65705,"56":-116.85,"57":-1.3722731,"58":88.8024,"59":-85.52879,"60":-65.09857,"61":-107.900696,"62":20.565252,"63":-68.77857,"64":52.128918,"65":96.40